In [0]:
import os
import json
import gzip
import pandas as pd
from urllib.request import urlopen

In [5]:
# !wget http://deepyeti.ucsd.edu/jianmo/amazon/sample/meta_Computers.json.
#!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/Electronics_5.json.gz
!wget https://drive.google.com/file/d/18wfGKYb0dgc6sUQn9Szylc6Qn5gPHdkN

--2020-03-22 03:16:51--  https://drive.google.com/file/d/18wfGKYb0dgc6sUQn9Szylc6Qn5gPHdkN/view?usp=sharing
Resolving drive.google.com (drive.google.com)... 172.217.193.102, 172.217.193.138, 172.217.193.113, ...
Connecting to drive.google.com (drive.google.com)|172.217.193.102|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘view?usp=sharing’

view?usp=sharing        [ <=>                ]  67.19K  --.-KB/s    in 0.006s  

2020-03-22 03:16:51 (11.2 MB/s) - ‘view?usp=sharing’ saved [68800]



In [3]:
data = []
with gzip.open('Electronics_5.json.gz') as f:
    for l in f:
        data.append(json.loads(l.strip()))
    
# total length of list, this number equals total number of products
print(len(data))

# first row of the list
print(data[0])

6739590
{'overall': 5.0, 'vote': '67', 'verified': True, 'reviewTime': '09 18, 1999', 'reviewerID': 'AAP7PPBU72QFM', 'asin': '0151004714', 'style': {'Format:': ' Hardcover'}, 'reviewerName': 'D. C. Carrad', 'reviewText': 'This is the best novel I have read in 2 or 3 years.  It is everything that fiction should be -- beautifully written, engaging, well-plotted and structured.  It has several layers of meanings -- historical, family,  philosophical and more -- and blends them all skillfully and interestingly.  It makes the American grad student/writers\' workshop "my parents were  mean to me and then my professors were mean to me" trivia look  childish and silly by comparison, as they are.\nAnyone who says this is an  adolescent girl\'s coming of age story is trivializing it.  Ignore them.  Read this book if you love literature.\nI was particularly impressed with  this young author\'s grasp of the meaning and texture of the lost world of  French Algeria in the 1950\'s and \'60\'s...parti

In [6]:
# convert list into pandas dataframe
df = pd.DataFrame.from_dict(data)

print(len(df))

6739590


In [0]:
### remove rows with unformatted title (i.e. some 'title' may still contain html style content)


df3 = df.fillna('')

df4 = df3[df3.reviewText.str.contains('var ')] # unformatted rows
df5 = df3[~df3.title.str.contains('var ')] # filter those unformatted rows

df4
print(len(df4))
print(len(df5))

KeyboardInterrupt: ignored

In [6]:
# how those unformatted rows look like
df4.iloc[0]

description                                                      
title           var aPageStart = (new Date()).getTime();\nvar ...
image                                                            
brand                                              DS Miller Inc.
rank            [>#1,826,312 in Electronics (See Top 100 in El...
main_cat                                                Computers
date                                                March 1, 2010
asin                                                   B0016C5EXY
feature                                                          
tech1                                                            
also_buy                                                         
price                                                            
also_view                                                        
tech2                                                            
details                                                          
similar_it